# Dataset Loading

In [1]:
PATH_TO_DATASET_FOLDER = "C:\\Users\\Vojta\\Desktop\\diploma\\data"

In [2]:
from enum import Enum
import os.path
import tensorflow as tf

In [3]:
class DataSet(Enum):
    Gutenberg = "gutenberg"

In [4]:
class DataSetType(Enum):
    Sentence = "Sentence"
    Article = "Article"

In [5]:
DATA_NAME = 'data.csv'

In [6]:
def create_path(directory, dataset, dataset_type, k=None):
    is_sentence_type = dataset_type == DataSetType.Sentence 
    if is_sentence_type and k is None:
        raise Exception(f"Sentence should be specified with k argument!")
    
    return os.path.join(directory, dataset.value, dataset_type.value + str(k), DATA_NAME) if is_sentence_type else os.path.join(directory, dataset.value, dataset_type.value, DATA_NAME)

In [7]:
create_path(PATH_TO_DATASET_FOLDER, DataSet.Gutenberg, DataSetType.Sentence, 10)

'C:\\Users\\Vojta\\Desktop\\diploma\\data\\gutenberg\\Sentence10\\data.csv'

In [8]:
def process_text(text):
    return text.lower()

In [9]:
def get_dataset_object_from_path(csv_filename, delim, text_pipeline_func=None):
    dataset = tf.data.TextLineDataset(filenames=csv_filename)
    
    def parse_csv(line):
        csv_line = bytes.decode(line.numpy())
        text, author = csv_line.split(delim)
        if text_pipeline_func is not None:
            text = text_pipeline_func(text)
        return text, author 

    dataset = dataset.map(lambda tpl: tf.py_function(parse_csv, [tpl], [tf.string, tf.string]))
    return dataset

In [10]:
ds = get_dataset_object_from_path(create_path(PATH_TO_DATASET_FOLDER, DataSet.Gutenberg, DataSetType.Sentence, 3), ';', process_text)

In [13]:
for line in ds.batch(8):
    pass
    print(line)
    break

(<tf.Tensor: shape=(8,), dtype=string, numpy=
array([b"the tragedy of pudd'nhead wilson  by mark twain     a whisper to the reader       _there is no character, howsoever good and fine, but it can      be destroyed by ridicule, howsoever poor and witless. observe the ass, for instance: his character is about      perfect, he is the choicest spirit among all the humbler      animals, yet see what ridicule has brought him to. instead      of feeling complimented when we are called an ass, we are      left in doubt._ --pudd'nhead wilson's calendar  a person who is ignorant of legal matters is always liable to make mistakes when he tries to photograph a court scene with his pen and so i was not willing to let the law chapters in this book go to press without first subjecting them to rigid and exhausting revision and correction by a trained barrister--if that is what they are called.",
       b"these chapters are right, now, in every detail, for they were rewritten under the immediate eye o